In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import math
import numpy as np
import pandas as pd
import glob
import time

from sklearn import linear_model, model_selection, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import OneClassSVM
from sklearn import model_selection, metrics
from scipy import stats # for statistical tests

import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive

plt.style.use('seaborn')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# Get the dataset
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
data = pd.read_csv('gdrive/My Drive/scraped_output_cleaned.csv', header=0, index_col=None)
data.head()

,Unnamed: 0,Review,Date,Stars,Restaurant
0,33,- Lobster Roll: Still one of my favourite lobs...,5 months ago,4 stars,TMG
1,0,"(Translated by Google) Good hamburgers, delici...",5 days ago,4 stars,TFDB
2,0,A great place to celebrate for event celebrati...,3 days ago,5 stars,Art
3,0,絕對是新加坡最好的法餐!\n完全洗刷掉之前在台灣吃其客座的平庸印象.\n果然主場才是最好發揮...,2 days ago,5 stars,Odette
4,0,Great food & atmosphere. Funny chefs and staff...,a week ago,5 stars,RIbid


In [6]:
reviews = data['Review']

In [7]:
reviews

0       - Lobster Roll: Still one of my favourite lobs...
1       (Translated by Google) Good hamburgers, delici...
2       A great place to celebrate for event celebrati...
3       絕對是新加坡最好的法餐!\n完全洗刷掉之前在台灣吃其客座的平庸印象.\n果然主場才是最好發揮...
4       Great food & atmosphere. Funny chefs and staff...
                              ...                        
2789                     Can’t get doneness of beef right
2790    love the burgers :) one of the best I've tried...
2791    Food selection isn't that comprehensive compar...
2792    Very average. Would have expected more for tha...
2793    Its a bit over rated. Seatings abit limited. T...
Name: Review, Length: 2794, dtype: object

In [6]:

reviews.to_numpy()

array(['- Lobster Roll: Still one of my favourite lobster roll places, and it’s still yummy like how I remembered it to be. Huge chunks of lobster and I love the bun. The curly fries and salad on the side were good too. …',
       '(Translated by Google) Good hamburgers, delicious pale ale and great cocktails.\n\n(Original) …',
       'A great place to celebrate for event celebration. Food and service were good. Value for money',
       ...,
       'Food selection isn\'t that comprehensive compared to other burger chains, and didn\'t really like the size of their burgers. Had to resort to using fork and spoon to "deconstruct" and eat the burger. The bread also breaks easily.  Non alcoholic drinks also terribly overpriced.',
       'Very average. Would have expected more for that price',
       'Its a bit over rated. Seatings abit limited. The seats i was at was squeezy. Burger buns was huge, more bun than meat. Beef was alright, overpowered by the taste of the cheeses. I got one from t

In [7]:
reviews[5]

'I have been to Hashida Singapore a couple of times now and I am so happy I discovered this place. The food is so authentic and the staff are very welcoming.'

In [9]:
arr = []
for i in reviews:
  i = i.replace("[","")
  i = i.replace("]","")
  i = i.replace("'","")
  words = nltk.word_tokenize(i)
  for j in words:
    if j not in arr:
      arr.append(j)
  #words = nltk.word_tokenize(sentence)
  #new_words= [word for word in words if word.isalnum()]

In [10]:
arr

['-',
 'Lobster',
 'Roll',
 ':',
 'Still',
 'one',
 'of',
 'my',
 'favourite',
 'lobster',
 'roll',
 'places',
 ',',
 'and',
 'it',
 '’',
 's',
 'still',
 'yummy',
 'like',
 'how',
 'I',
 'remembered',
 'to',
 'be',
 '.',
 'Huge',
 'chunks',
 'love',
 'the',
 'bun',
 'The',
 'curly',
 'fries',
 'salad',
 'on',
 'side',
 'were',
 'good',
 'too',
 '…',
 '(',
 'Translated',
 'by',
 'Google',
 ')',
 'Good',
 'hamburgers',
 'delicious',
 'pale',
 'ale',
 'great',
 'cocktails',
 'Original',
 'A',
 'place',
 'celebrate',
 'for',
 'event',
 'celebration',
 'Food',
 'service',
 'Value',
 'money',
 '絕對是新加坡最好的法餐',
 '!',
 '完全洗刷掉之前在台灣吃其客座的平庸印象',
 '果然主場才是最好發揮主廚光芒的地方',
 'Great',
 'food',
 '&',
 'atmosphere',
 'Funny',
 'chefs',
 'staffs',
 'that',
 'interacting',
 'well',
 'with',
 'patrons',
 'Can',
 'see',
 'passions',
 'in',
 'their',
 'eyes',
 'have',
 'been',
 'Hashida',
 'Singapore',
 'a',
 'couple',
 'times',
 'now',
 'am',
 'so',
 'happy',
 'discovered',
 'this',
 'is',
 'authentic',
 'staff'

In [41]:
tagged = nltk.pos_tag(arr)

In [42]:
tagged

[('-', ':'),
 ('Lobster', 'NN'),
 ('Roll', 'NN'),
 (':', ':'),
 ('Still', 'RB'),
 ('one', 'CD'),
 ('of', 'IN'),
 ('my', 'PRP$'),
 ('favourite', 'JJ'),
 ('lobster', 'NN'),
 ('roll', 'NN'),
 ('places', 'NNS'),
 (',', ','),
 ('and', 'CC'),
 ('it', 'PRP'),
 ('’', 'NNP'),
 ('s', 'VBD'),
 ('still', 'RB'),
 ('yummy', 'VBN'),
 ('like', 'IN'),
 ('how', 'WRB'),
 ('I', 'PRP'),
 ('remembered', 'VBD'),
 ('to', 'TO'),
 ('be', 'VB'),
 ('.', '.'),
 ('Huge', 'JJ'),
 ('chunks', 'NNS'),
 ('love', 'VBP'),
 ('the', 'DT'),
 ('bun', 'NN'),
 ('The', 'DT'),
 ('curly', 'JJ'),
 ('fries', 'NNS'),
 ('salad', 'VBP'),
 ('on', 'IN'),
 ('side', 'NN'),
 ('were', 'VBD'),
 ('good', 'JJ'),
 ('too', 'RB'),
 ('…', 'JJ'),
 ('(', '('),
 ('Translated', 'VBN'),
 ('by', 'IN'),
 ('Google', 'NNP'),
 (')', ')'),
 ('Good', 'NNP'),
 ('hamburgers', 'NNS'),
 ('delicious', 'JJ'),
 ('pale', 'JJ'),
 ('ale', 'JJ'),
 ('great', 'JJ'),
 ('cocktails', 'NNS'),
 ('Original', 'NNP'),
 ('A', 'NNP'),
 ('place', 'NN'),
 ('celebrate', 'NN'),
 ('for',

In [43]:
tagged = pd.DataFrame(tagged)

In [44]:
tagged

,0,1
0,-,:
1,Lobster,NN
2,Roll,NN
3,:,:
4,Still,RB
...,...,...
6467,Seatings,NNP
6468,squeezy,NN
6469,cheeses,NNS
6470,Saving,VBG


In [45]:
tagged.rename(columns={0:'word',1:'tag'}, inplace=True)

In [47]:
from google.colab import files
tagged.to_csv('tagged_new.csv', index=False) 
files.download('tagged_new.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Building

In [49]:
data = pd.read_csv('gdrive/My Drive/tagged_new.csv', header=0, index_col=None)
data.head()

,word,tag
0,-,:
1,Lobster,NN
2,Roll,NN
3,:,:
4,Still,RB


In [51]:
from csv import reader
with open('gdrive/My Drive/tagged_new.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Get all rows of csv from csv_reader object as list of tuples
    list_of_tuples = list(map(tuple, csv_reader))
    # display all rows of csv
    print(list_of_tuples)

[('word', 'tag'), ('-', ':'), ('Lobster', 'NN'), ('Roll', 'NN'), (':', ':'), ('Still', 'RB'), ('one', 'CD'), ('of', 'IN'), ('my', 'PRP$'), ('favourite', 'JJ'), ('lobster', 'NN'), ('roll', 'NN'), ('places', 'NNS'), (',', ','), ('and', 'CC'), ('it', 'PRP'), ('’', 'NNP'), ('s', 'VBD'), ('still', 'RB'), ('yummy', 'VBN'), ('like', 'IN'), ('how', 'WRB'), ('I', 'PRP'), ('remembered', 'VBD'), ('to', 'TO'), ('be', 'VB'), ('.', '.'), ('Huge', 'JJ'), ('chunks', 'NNS'), ('love', 'VBP'), ('the', 'DT'), ('bun', 'NN'), ('The', 'DT'), ('curly', 'JJ'), ('fries', 'NNS'), ('salad', 'VBP'), ('on', 'IN'), ('side', 'NN'), ('were', 'VBD'), ('good', 'JJ'), ('too', 'RB'), ('…', 'JJ'), ('(', '('), ('Translated', 'VBN'), ('by', 'IN'), ('Google', 'NNP'), (')', ')'), ('Good', 'NNP'), ('hamburgers', 'NNS'), ('delicious', 'JJ'), ('pale', 'JJ'), ('ale', 'JJ'), ('great', 'JJ'), ('cocktails', 'NNS'), ('Original', 'NNP'), ('A', 'NNP'), ('place', 'NN'), ('celebrate', 'NN'), ('for', 'IN'), ('event', 'NN'), ('celebration',

# Start from here

In [8]:
from nltk import word_tokenize, pos_tag
lst = []
for i in reviews:
  lst.append(pos_tag(word_tokenize(i)))

In [9]:
lst

[[('-', ':'),
  ('Lobster', 'NN'),
  ('Roll', 'NN'),
  (':', ':'),
  ('Still', 'RB'),
  ('one', 'CD'),
  ('of', 'IN'),
  ('my', 'PRP$'),
  ('favourite', 'JJ'),
  ('lobster', 'NN'),
  ('roll', 'NN'),
  ('places', 'NNS'),
  (',', ','),
  ('and', 'CC'),
  ('it', 'PRP'),
  ('’', 'NNP'),
  ('s', 'VBD'),
  ('still', 'RB'),
  ('yummy', 'VBN'),
  ('like', 'IN'),
  ('how', 'WRB'),
  ('I', 'PRP'),
  ('remembered', 'VBD'),
  ('it', 'PRP'),
  ('to', 'TO'),
  ('be', 'VB'),
  ('.', '.'),
  ('Huge', 'JJ'),
  ('chunks', 'NNS'),
  ('of', 'IN'),
  ('lobster', 'NN'),
  ('and', 'CC'),
  ('I', 'PRP'),
  ('love', 'VBP'),
  ('the', 'DT'),
  ('bun', 'NN'),
  ('.', '.'),
  ('The', 'DT'),
  ('curly', 'JJ'),
  ('fries', 'NNS'),
  ('and', 'CC'),
  ('salad', 'NN'),
  ('on', 'IN'),
  ('the', 'DT'),
  ('side', 'NN'),
  ('were', 'VBD'),
  ('good', 'JJ'),
  ('too', 'RB'),
  ('.', '.'),
  ('…', 'VB')],
 [('(', '('),
  ('Translated', 'VBN'),
  ('by', 'IN'),
  ('Google', 'NNP'),
  (')', ')'),
  ('Good', 'NNP'),
  ('hambu

In [10]:
tagged_data = lst

In [11]:
tagged_data[1]

[('(', '('),
 ('Translated', 'VBN'),
 ('by', 'IN'),
 ('Google', 'NNP'),
 (')', ')'),
 ('Good', 'NNP'),
 ('hamburgers', 'NNS'),
 (',', ','),
 ('delicious', 'JJ'),
 ('pale', 'JJ'),
 ('ale', 'NN'),
 ('and', 'CC'),
 ('great', 'JJ'),
 ('cocktails', 'NNS'),
 ('.', '.'),
 ('(', '('),
 ('Original', 'NNP'),
 (')', ')'),
 ('…', 'NN')]

In [12]:
def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'has_hyphen': '-' in sentence[index],
        'is_numeric': sentence[index].isdigit(),
        'capitals_inside': sentence[index][1:].lower() != sentence[index][1:]
    }
 
import pprint 
pprint.pprint(features(['This', 'is', 'a', 'sentence'], 2))
 
{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}
 

{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}


{'capitals_inside': False,
 'has_hyphen': False,
 'is_all_caps': False,
 'is_all_lower': True,
 'is_capitalized': False,
 'is_first': False,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prefix-2': 'a',
 'prefix-3': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'suffix-2': 'a',
 'suffix-3': 'a',
 'word': 'a'}

In [13]:

def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [14]:
cutoff = int(.75 * len(tagged_data))
training_data = tagged_data[:cutoff]
test_data = tagged_data[cutoff:]
 
print(len(training_data))  
print(len(test_data))     
 

def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_data:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X_train, y_train = transform_to_dataset(training_data)
X_test, y_test = transform_to_dataset(test_data)
 

2095
699


In [15]:
X_train[0]

{'capitals_inside': False,
 'has_hyphen': True,
 'is_all_caps': True,
 'is_all_lower': True,
 'is_capitalized': True,
 'is_first': True,
 'is_last': False,
 'is_numeric': False,
 'next_word': 'Lobster',
 'prefix-1': '-',
 'prefix-2': '-',
 'prefix-3': '-',
 'prev_word': '',
 'suffix-1': '-',
 'suffix-2': '-',
 'suffix-3': '-',
 'word': '-'}

In [16]:
len(X_train)

80390

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
 
clf.fit(X_train[:15000], y_train[:15000])   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
 
#X_test, y_test = transform_to_dataset(test_sentences)

print("Accuracy:", clf.score(X_test, y_test))
 

Accuracy: 0.9126508272173156


In [18]:
def pos_tag(sentence):
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

In [19]:
new_data = pos_tag(word_tokenize('The food here is amazing, would recommend for sure'))

In [20]:
list(zip(new_data))

[(('The', 'DT'),),
 (('food', 'NN'),),
 (('here', 'RB'),),
 (('is', 'VBZ'),),
 (('amazing', 'JJ'),),
 ((',', ','),),
 (('would', 'MD'),),
 (('recommend', 'VB'),),
 (('for', 'IN'),),
 (('sure', 'NN'),)]

In [21]:
old_data = pos_tag(word_tokenize('food was tasty and larger portions so much lobster in the lobster roll but service was a bit slow'))

In [22]:
list(zip(old_data))

[(('food', 'NN'),),
 (('was', 'VBD'),),
 (('tasty', 'JJ'),),
 (('and', 'CC'),),
 (('larger', 'JJR'),),
 (('portions', 'NNS'),),
 (('so', 'RB'),),
 (('much', 'JJ'),),
 (('lobster', 'NN'),),
 (('in', 'IN'),),
 (('the', 'DT'),),
 (('lobster', 'NN'),),
 (('roll', 'NN'),),
 (('but', 'CC'),),
 (('service', 'NN'),),
 (('was', 'VBD'),),
 (('a', 'DT'),),
 (('bit', 'NN'),),
 (('slow', 'JJ'),)]

In [ ]:
# List of tags: https://cs.nyu.edu/~grishman/jet/guide/PennPOS.html